<a href="https://colab.research.google.com/github/PritamMoitra/Learning_Python_for_Finance/blob/main/ADXStochasBTMultiSec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import time
from ta.trend import adx

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=21c92e6ac7512dc037a522fa1e3f123bbcf2cee18e4d7603b975627caa53c15d
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [ ]:
# Parameters
tickers = [ 'APOLLOHOSP.NS','ASIANPAINT.NS','AXISBANK.NS',
            'BAJAJ-AUTO.NS','BAJFINANCE.NS','BAJAJFINSV.NS','BEL.NS','BHARTIARTL.NS',
            'CIPLA.NS','COALINDIA.NS','DRREDDY.NS','EICHERMOT.NS','ETERNAL.NS',
            'GRASIM.NS','HCLTECH.NS','HDFCBANK.NS','HDFCLIFE.NS','HEROMOTOCO.NS',
            'HINDALCO.NS','HINDUNILVR.NS','ICICIBANK.NS','ITC.NS','INDUSINDBK.NS',
            'INFY.NS','JSWSTEEL.NS','JIOFIN.NS','KOTAKBANK.NS','LT.NS',
            'M&M.NS','MARUTI.NS','NTPC.NS','NESTLEIND.NS','ONGC.NS',
            'POWERGRID.NS','RELIANCE.NS','SBILIFE.NS','SHRIRAMFIN.NS','SBIN.NS',
            'SUNPHARMA.NS','TCS.NS','TATACONSUM.NS','TATAMOTORS.NS','TATASTEEL.NS',
            'TECHM.NS','TITAN.NS','TRENT.NS','ULTRACEMCO.NS','WIPRO.NS','ADANIENT.NS','ADANIPORTS.NS']
start_date = '2025-06-19'
end_date = '2025-06-20'
adx_threshold = 20
initial_capital = 52674.06
k_period = 14
d_period = 3
target_pct = 0.02
stoploss_pct = 0.01

In [ ]:
def fetch_intraday_data(ticker, start_date, end_date, interval="1m"):
    df = yf.download(ticker, start=start_date, end=end_date, interval=interval, progress=False, auto_adjust=False)
    print(df.tail())
    df.index = pd.to_datetime(df.index)
    df.index = df.index.tz_convert('Asia/Kolkata')

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    # Remove any remaining multi-dimensional data
    df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
    return df

In [ ]:
def is_trending(df, adx_period=14):
    if len(df) < adx_period + 1:
        return False
    adx_series = adx(df['High'], df['Low'], df['Close'], window=adx_period)
    latest_adx = adx_series.dropna().iloc[-1] if not adx_series.dropna().empty else 0
    return latest_adx > adx_threshold

In [ ]:
def run_strategy(df):
    capital = initial_capital
    in_position = False
    trade_log = []
    minute_data = pd.DataFrame()
    entry_price = qty = 0
    position_type = ''

    opening_price = df.iloc[0]['Open']

    for timestamp, row in df.iterrows():
        minute_data = pd.concat([minute_data, pd.DataFrame([row], index=[timestamp])])

        if len(minute_data) < k_period + d_period:
            continue

        trend = 'uptrend' if row['Close'] > opening_price else 'downtrend'

        recent = minute_data.iloc[-(k_period + d_period):].copy()
        low_min = recent['Low'].rolling(k_period).min()
        high_max = recent['High'].rolling(k_period).max()
        recent['%K'] = 100 * (recent['Close'] - low_min) / (high_max - low_min)
        recent['%D'] = recent['%K'].rolling(d_period).mean()

        row_prev = recent.iloc[-2]
        row_curr = recent.iloc[-1]

        # Exit logic
        if in_position:
            target_price = entry_price * (1 + target_pct) if position_type == 'long' else entry_price * (1 - target_pct)
            stoploss_price = entry_price * (1 - stoploss_pct) if position_type == 'long' else entry_price * (1 + stoploss_pct)

            if (position_type == 'long' and (row['High'] >= target_price or row['Low'] <= stoploss_price)) or \
               (position_type == 'short' and (row['Low'] <= target_price or row['High'] >= stoploss_price)):
                exit_price = target_price if (row['High'] >= target_price or row['Low'] <= target_price) else stoploss_price
                profit = (exit_price - entry_price) * qty if position_type == 'long' else (entry_price - exit_price) * qty
                capital += profit
                trade_log.append({
                    'Type': position_type,
                    'Entry': entry_price,
                    'Exit': exit_price,
                    'Qty': qty,
                    'PnL': profit,
                    'Entry Time': entry_time,
                    'Exit Time': timestamp
                })
                in_position = False

        # Entry logic
        if not in_position:
            if trend == 'uptrend' and row_prev['%K'] < row_prev['%D'] and row_curr['%K'] > row_curr['%D'] and row_curr['%K'] < 20:
                entry_price = row['Close']
                qty = capital // entry_price
                entry_time = timestamp
                in_position = True
                position_type = 'long'
            elif trend == 'downtrend' and row_prev['%K'] > row_prev['%D'] and row_curr['%K'] < row_curr['%D'] and row_curr['%K'] > 80:
                entry_price = row['Close']
                qty = capital // entry_price
                entry_time = timestamp
                in_position = True
                position_type = 'short'

        # EOD Exit
        if timestamp.time() >= time(15, 15) and in_position:
            exit_price = row['Close']
            profit = (exit_price - entry_price) * qty if position_type == 'long' else (entry_price - exit_price) * qty
            capital += profit
            trade_log.append({
                'Type': position_type,
                'Entry': entry_price,
                'Exit': exit_price,
                'Qty': qty,
                'PnL': profit,
                'Entry Time': entry_time,
                'Exit Time': timestamp
            })
            in_position = False
            break

    return trade_log, capital

In [ ]:
def summarize_trades(trade_log, capital):
    wins = sum(1 for t in trade_log if t['PnL'] > 0)
    losses = len(trade_log) - wins

    print("\n📘 Trade Summary:")
    for i, t in enumerate(trade_log, 1):
        print(f"{i}) {t['Type'].capitalize()} | Entry: ₹{t['Entry']:.2f} at {t['Entry Time']} | Exit: ₹{t['Exit']:.2f} at {t['Exit Time']} | Qty: {t['Qty']} | P&L: ₹{t['PnL']:.2f}")

    print(f"\n✅ Wins: {wins} | ❌ Losses: {losses}")
    print(f"💰 Final Capital: ₹{capital:.2f}")

In [ ]:
# --- Main execution ---
if __name__ == "__main__":
    selected_ticker = None
    for ticker in tickers:
        df = fetch_intraday_data(ticker, start_date, end_date)
        if is_trending(df):
            selected_ticker = ticker
            print(f"✅ Selected Trending Ticker: {ticker}")
            trades, final_capital = run_strategy(df)
            summarize_trades(trades, final_capital)
            break

    if not selected_ticker:
        print("❌ No trending ticker found.")

Price                         Adj Close         Close          High  \
Ticker                    APOLLOHOSP.NS APOLLOHOSP.NS APOLLOHOSP.NS   
Datetime                                                              
2025-06-19 09:55:00+00:00        6992.5        6992.5        6995.0   
2025-06-19 09:56:00+00:00        6992.0        6992.0        6997.0   
2025-06-19 09:57:00+00:00        6994.5        6994.5        6994.5   
2025-06-19 09:58:00+00:00        6992.0        6992.0        6995.0   
2025-06-19 09:59:00+00:00        6996.0        6996.0        6996.0   

Price                               Low          Open        Volume  
Ticker                    APOLLOHOSP.NS APOLLOHOSP.NS APOLLOHOSP.NS  
Datetime                                                             
2025-06-19 09:55:00+00:00        6991.5        6993.5          1689  
2025-06-19 09:56:00+00:00        6992.0        6992.5          1394  
2025-06-19 09:57:00+00:00        6992.0        6992.5          1237  
2025-06-19 